## Setup and Data Import

In [1]:
import sys
sys.path.insert(0, '..')

from joblib import load

import Functions as fxns
from Sita_Functions import np, pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

from datetime import timedelta

## Claims DF

In [2]:
# !python ../Preprocessing.py

In [3]:
claims = load('../claims.pkl')

### New Columns

In [4]:
claims['AgeAtService'] = ((claims.ClaimStartDt - claims.DOB)
                              / timedelta(days=365)).astype(int)
claims['HasDied'] = claims.DOD.notna()

physician_cols = \
    claims.columns[claims.columns.str.contains('Physician')].to_list()
claims['HasAnyPhysician'] = claims[physician_cols].notna().any(axis=1)
claims['HasAllPhysicians'] = claims[physician_cols].notna().all(axis=1)

claims['ClaimDuration'] = \
    (claims.ClaimEndDt  - claims.ClaimStartDt).dt.days + 1
claims['IPDuration'] = \
    (claims.DischargeDt - claims.AdmissionDt).dt.days + 1

claims['ClaimCost'] = \
    claims.InscClaimAmtReimbursed + claims.DeductibleAmtPaid
claims['PercInsCovered'] = \
    round((claims.InscClaimAmtReimbursed / claims.ClaimCost) * 100)
claims['DailyClaimCost'] = \
    round(claims.ClaimCost / claims.ClaimDuration)

# add date cols containing only day, week, year
fxns.split_date(claims, ['ClaimStartDt', 'ClaimEndDt', 'AdmissionDt', 'DischargeDt'])

# add claims per beneficiary col
claims_per_bene = claims.groupby('BeneID').count()[['ClaimID']]
claims_per_bene.columns = ['ClaimsPerBene']
claims = pd.merge(claims, claims_per_bene, how='left', left_on='BeneID', right_on=claims_per_bene.index)
# ratio inpatient claims

### Pre-processing

In [5]:
# encode bool colums to 0/1
fxns.re_encode_bool(claims, ['HasDied', 'HasAnyPhysician', 'HasAllPhysicians'])

In [6]:
# drop ChronicCond_ prefix from applicable cols
fxns.drop_chronic_prefix(claims)

In [7]:
# change object cols to dtype category
fxns.to_category_dtype(claims)

### Variables

In [8]:
numeric_cols     = fxns.cols_by_dtype(claims)[0]
categorical_cols = fxns.cols_by_dtype(claims)[1]
date_cols        = fxns.cols_by_dtype(claims)[2]

physician_cols = \
    claims.columns[claims.columns.str.contains('Physician')].to_list()
chronic_cols   = \
    claims.columns[claims.columns.str.contains('Chronic')].to_list()
diagnosis_cols = \
    claims.columns[claims.columns.str.contains('Diagnosis')].to_list()
procedure_cols = \
    claims.columns[claims.columns.str.contains('Procedure')].to_list()

### Summary Data

In [9]:
outpatient_claims = claims.loc[claims.IsOutpatient == 1]
inpatient_claims  = claims.loc[claims.IsOutpatient == 0]

In [10]:
groupby_providers   = claims.groupby('Provider')

In [11]:
mean_by_provider    = \
    groupby_providers.mean().add_suffix('_mean').reset_index()

In [12]:
sum_by_provider     = \
    groupby_providers.sum().add_suffix('_sum').reset_index()

In [13]:
chronic_by_provider = claims[['Provider'] + chronic_cols
                            ].groupby('Provider').sum().reset_index()

In [14]:
# need to get unique counts
count_by_provider = groupby_providers[
    ['BeneID', 'ClaimID', 'County', 'State', 'IPDuration']
    + physician_cols + diagnosis_cols + procedure_cols
    ].count().add_suffix('_count').reset_index()

# ADD PHYSICIAN RATIOS, SUMS OF EACH
# CLAIM ADMIT CODE, DIAGNOSIS GROUP CODE?
# ADD RANGE OF AGE

In [15]:
providers = pd.merge(mean_by_provider, sum_by_provider, on='Provider')
providers = pd.merge(providers, count_by_provider, on='Provider')
providers = pd.merge(providers, chronic_by_provider, on='Provider')

In [16]:
providers.sort_index(axis=1, inplace=True)

In [17]:
# print(claims.columns, '\n')

# Can't use, but have IPDuration count/mean/sum:
#     'AdmissionDt', 'AdmissionDt_Week',
#     'DischargeDt', 'DischargeDt_Week'

# Can't use, but have ClaimDuration count/mean/sum:
#     'ClaimEndDt', 'ClaimEndDt_Week', 'ClaimID'
#     'ClaimStartDt', 'ClaimStartDt_Week',


# Can't use, but have AgeAtService mean/sum and HasDied sum/ratio:
#     'DOB', 'DOD'

In [18]:
print(providers.shape, '\n')
print(providers.columns)
providers.sample(5)

(5410, 70) 

Index(['AgeAtService_mean', 'AgeAtService_sum', 'Alzheimers_Chronic',
       'AttendingPhysician_count', 'BeneID_count', 'Cancer_Chronic',
       'ClaimCost_mean', 'ClaimCost_sum', 'ClaimDuration_mean',
       'ClaimDuration_sum', 'ClaimID_count', 'ClaimsPerBene_mean',
       'ClaimsPerBene_sum', 'ClmAdmitDiagnosisCode_count',
       'ClmDiagnosisCode_10_count', 'ClmDiagnosisCode_1_count',
       'ClmDiagnosisCode_2_count', 'ClmDiagnosisCode_3_count',
       'ClmDiagnosisCode_4_count', 'ClmDiagnosisCode_5_count',
       'ClmDiagnosisCode_6_count', 'ClmDiagnosisCode_7_count',
       'ClmDiagnosisCode_8_count', 'ClmDiagnosisCode_9_count',
       'ClmProcedureCode_1_count', 'ClmProcedureCode_2_count',
       'ClmProcedureCode_3_count', 'ClmProcedureCode_4_count',
       'ClmProcedureCode_5_count', 'ClmProcedureCode_6_count', 'County_count',
       'DailyClaimCost_mean', 'DailyClaimCost_sum', 'DeductibleAmtPaid_mean',
       'DeductibleAmtPaid_sum', 'Depression_Chronic', 'Diab

,AgeAtService_mean,AgeAtService_sum,Alzheimers_Chronic,AttendingPhysician_count,BeneID_count,Cancer_Chronic,ClaimCost_mean,ClaimCost_sum,ClaimDuration_mean,ClaimDuration_sum,ClaimID_count,ClaimsPerBene_mean,ClaimsPerBene_sum,ClmAdmitDiagnosisCode_count,ClmDiagnosisCode_10_count,ClmDiagnosisCode_1_count,ClmDiagnosisCode_2_count,ClmDiagnosisCode_3_count,ClmDiagnosisCode_4_count,ClmDiagnosisCode_5_count,ClmDiagnosisCode_6_count,ClmDiagnosisCode_7_count,ClmDiagnosisCode_8_count,ClmDiagnosisCode_9_count,ClmProcedureCode_1_count,ClmProcedureCode_2_count,ClmProcedureCode_3_count,ClmProcedureCode_4_count,ClmProcedureCode_5_count,ClmProcedureCode_6_count,County_count,DailyClaimCost_mean,DailyClaimCost_sum,DeductibleAmtPaid_mean,DeductibleAmtPaid_sum,Depression_Chronic,Diabetes_Chronic,DiagnosisGroupCode_count,HasAllPhysicians_count,HasAnyPhysician_count,HeartFailure_Chronic,IPAnnualDeductibleAmt_mean,IPAnnualDeductibleAmt_sum,IPAnnualReimbursementAmt_mean,IPAnnualReimbursementAmt_sum,IPDuration_count,IPDuration_mean,IPDuration_sum,InscClaimAmtReimbursed_mean,InscClaimAmtReimbursed_sum,IschemicHeart_Chronic,KidneyDisease_Chronic,NoOfMonths_PartACov_mean,NoOfMonths_PartACov_sum,NoOfMonths_PartBCov_mean,NoOfMonths_PartBCov_sum,OPAnnualDeductibleAmt_mean,OPAnnualDeductibleAmt_sum,OPAnnualReimbursementAmt_mean,OPAnnualReimbursementAmt_sum,ObstrPulmonary_Chronic,OperatingPhysician_count,Osteoporosis_Chronic,OtherPhysician_count,PercInsCovered_mean,PercInsCovered_sum,Provider,RheumatoidArthritis_Chronic,State_count,Stroke_Chronic
2271,71.937500,1151,5,16,16,0,380.625000,6090.0,2.937500,47,16,4.750000,76,4,0,15,8,6,2,1,0,0,0,0,0,0,0,0,0,0,16,306.312500,4901.0,11.250000,180.0,5,9,0,16,16,6,267.000000,4272,1437.500000,23000,0,NaN,0.0,369.375000,5910,12,3,12.0,192,11.25,180,645.000000,10320,2822.500000,45160,0,5,3,6,91.312500,1461.0,PRV53829,5,16,0
4507,71.010417,6817,36,94,96,17,484.687500,46530.0,3.291667,316,96,7.312500,702,19,0,95,62,44,25,17,11,9,7,4,0,0,0,0,0,0,96,239.437500,22986.0,1.354167,130.0,46,69,0,96,96,49,400.500000,38448,3845.416667,369160,0,NaN,0.0,483.333333,46400,80,45,12.0,1152,12.00,1152,1152.187500,110610,4826.145833,463310,37,16,30,31,99.138298,9319.0,PRV56639,19,96,18
3039,66.000000,66,0,1,1,0,100.000000,100.0,1.000000,1,1,9.000000,9,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,100.000000,100.0,0.000000,0.0,0,1,0,1,1,0,0.000000,0,0.000000,0,0,NaN,0.0,100.000000,100,1,0,12.0,12,12.00,12,200.000000,200,3790.000000,3790,0,0,0,1,100.000000,100.0,PRV54799,0,1,0
202,76.238095,1601,6,21,21,0,7439.619048,156232.0,3.142857,66,21,7.047619,148,4,0,21,12,8,6,3,3,3,2,2,3,2,0,0,0,0,21,900.142857,18903.0,203.428571,4272.0,12,17,4,21,21,13,813.714286,17088,18880.000000,396480,4,7.25,29.0,7236.190476,151960,17,9,12.0,252,12.00,252,925.714286,19440,4005.238095,84110,4,4,7,10,94.571429,1986.0,PRV51259,5,21,0
3423,78.833333,473,4,6,6,0,365.000000,2190.0,1.000000,6,6,10.166667,61,1,0,6,2,2,1,1,1,0,0,0,0,0,0,0,0,0,6,365.000000,2190.0,0.000000,0.0,0,6,0,6,6,5,712.000000,4272,4753.333333,28520,0,NaN,0.0,365.000000,2190,6,5,12.0,72,12.00,72,1218.333333,7310,3050.000000,18300,1,3,2,3,100.000000,600.0,PRV55291,5,6,0


## New Columns

### Beneficiaries

In [19]:
providers['IPClaims_count'] = inpatient_claims.groupby('Provider').IsOutpatient.count().values
providers['OPClaims_count'] = outpatient_claims.groupby('Provider').IsOutpatient.count().values

# Ratio of inpatient claims can be determined from the above - do we need a separate col for it?

In [20]:
def binary_ratios(df, col, group1, group2):
    '''
    Adds a new column to a df that lists the ratio of one value from a binary column in the
    same df. (The ratio of the other value in the binary column is the inverse.)
    
    Arguments: A single dataframe.
    
    Output: None.
    
    Returns: Dataframe altered in place.
    '''
    by_provider = df.groupby(['Provider', col]).ClaimID.count().reset_index()

    colname = pd.DataFrame()
    colname[group1] = by_provider[by_provider[col] == 1].ClaimID.to_list()
    colname[group2] = by_provider[by_provider[col] == 0].ClaimID.to_list()
    
    newcolname1 = f'{group1}_ratio'
    
    providers[newcolname1] = round((colname[group1] / (colname[group1] + colname[group2])) * 100, 2)

In [21]:
binary_ratios(claims, 'Gender', 'GenderZero', 'GenderOne')
binary_ratios(claims, 'RenalDisease', 'HasRenalDisease', 'NotRenalDisease')
binary_ratios(claims, 'HasDied', 'HasDied', 'NotDead')

In [22]:
race_by_provider = claims.groupby(['Provider', 'Race']).ClaimID.count().reset_index()

race = pd.DataFrame()
race['race1'] = race_by_provider[race_by_provider.Race == 1].ClaimID.to_list()
race['race0'] = race_by_provider[race_by_provider.Race == 0].ClaimID.to_list()
race['race3'] = race_by_provider[race_by_provider.Race == 3].ClaimID.to_list()
race['race5'] = race_by_provider[race_by_provider.Race == 5].ClaimID.to_list()

providers['RaceZero_ratio']   = round((race.race0 / np.sum(race, axis=1) * 100), 2)
providers['RaceThree_ratio'] = round((race.race3 / np.sum(race, axis=1) * 100), 2)
providers['RaceFive_ratio']  = round((race.race5 / np.sum(race, axis=1) * 100), 2)

In [23]:
# DO WE WANT CHRONIC CONDITION RATIOS? WHY/WHY NOT?

In [24]:
# ratio_cols = providers.columns[providers.columns.str.contains('_ratio')].to_list()
# providers[['Provider'] + ratio_cols].sample(5)

### Inpatients/Outpatiens

In [25]:
binary_ratios(claims, 'IsOutpatient', 'IsOutpatient', 'Inpatient')

### Doctors

In [26]:
# providers['ClaimsPerBene'] = 


In [28]:
providers.columns

Index(['AgeAtService_mean', 'AgeAtService_sum', 'Alzheimers_Chronic',
       'AttendingPhysician_count', 'BeneID_count', 'Cancer_Chronic',
       'ClaimCost_mean', 'ClaimCost_sum', 'ClaimDuration_mean',
       'ClaimDuration_sum', 'ClaimID_count', 'ClaimsPerBene_mean',
       'ClaimsPerBene_sum', 'ClmAdmitDiagnosisCode_count',
       'ClmDiagnosisCode_10_count', 'ClmDiagnosisCode_1_count',
       'ClmDiagnosisCode_2_count', 'ClmDiagnosisCode_3_count',
       'ClmDiagnosisCode_4_count', 'ClmDiagnosisCode_5_count',
       'ClmDiagnosisCode_6_count', 'ClmDiagnosisCode_7_count',
       'ClmDiagnosisCode_8_count', 'ClmDiagnosisCode_9_count',
       'ClmProcedureCode_1_count', 'ClmProcedureCode_2_count',
       'ClmProcedureCode_3_count', 'ClmProcedureCode_4_count',
       'ClmProcedureCode_5_count', 'ClmProcedureCode_6_count', 'County_count',
       'DailyClaimCost_mean', 'DailyClaimCost_sum', 'DeductibleAmtPaid_mean',
       'DeductibleAmtPaid_sum', 'Depression_Chronic', 'Diabetes_Chronic'

### Codes

### Money

### Time

### Location

### Size